# AFL Data Scrape from afltables.com

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

## Getting all unique player urls from teams

In [2]:
# How teams appear in team url
team_url_list = ['adelaide', 'brisbanel', 'carlton', 'collingwood', 'essendon', 'fremantle', 
                 'geelong', 'goldcoast', 'gws', 'hawthorn', 'melbourne', 'kangaroos', 'padelaide',
                 'richmond', 'stkilda', 'swans', 'westcoast', 'bullldogs']

In [3]:
player_url_set = set()
for team in team_url_list:
    print(team)
    page = requests.get('https://afltables.com/afl/stats/teams/' + team + '.html')
    soup = BeautifulSoup(page.text)
    tables = soup.findAll("table")
#     print('team tables count:', len(tables))
#     print()
    if not len(tables):
        raise Exception('Missing Table') 
    for row in tables[0].tbody.findAll('tr'):
        try:
            href = row.find('a').get("href")
        except:
            href = None
            print('Error with row:')
            print(row)
            print()
        if not href:
            raise Exception('Missing href')
        player_url_set.add(href)

adelaide
brisbanel
carlton
collingwood
essendon
fremantle
geelong
goldcoast
gws
hawthorn
melbourne
kangaroos
padelaide
richmond
stkilda
swans
westcoast
bullldogs


In [4]:
len(player_url_set)

5477

## Getting all player data from each player url

In [5]:
players_data_list = []
for i, player_url in enumerate(player_url_set):
#     print(i, player_url)
    tables_list = pd.read_html('https://afltables.com/afl/stats' + player_url[2:])
    try:
        season_tables = [table for table in tables_list if isinstance(table.columns, pd.MultiIndex)]
    except Exception as e:
        print(e)
        print()
    if not season_tables:
        print('season table missing')
#     print(len(season_tables))
    players_data_list.append((player_url, season_tables))

## Creating player data

In [6]:
player_df = pd.DataFrame(columns=['name', 'afl_tables_url', 'team', 'year', 'game_num', 'opponent', 'Rd', 'R', 
                                  '#', 'KI', 'MK', 'HB', 'DI', 'GL', 'BH', 'HO', 'TK', 'RB', 'IF', 'CL', 
                                  'CG', 'FF', 'FA', 'BR', 'CP', 'UP', 'CM', 'MI', '1%', 'BO', 'GA', '%P'])
player_df

,name,afl_tables_url,team,year,game_num,opponent,Rd,R,#,KI,...,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P


In [7]:
def func(i, data):
    player_url, player_table_list = data[0], data[1]
#     print(i, player_url)
    player_name = player_url.split('/')[-1][:-5].replace('_', ' ')
    year_set = {int(season.columns[0][0].split('-')[1].strip()) > 2011 for season in player_table_list}
#     print(year_set)
    if not any(year_set):
        return
    for season in player_table_list:
        labels = [d.strip() for d in season.columns[0][0].split('-')]
        if len(labels) != 2:
            raise Exception(f'Error with {player_url}, {season.columns[0]}')
        team_name, year = labels

        for i, row in season.iterrows():
            created_row = [player_name, player_url, team_name, year] + list(row.values)
            player_df.loc[len(player_df)] = created_row

In [8]:
for i, data in enumerate(players_data_list):
    func(i, data)

In [9]:
player_df[['KI','MK','HB','DI','GL','BH','HO','TK','RB','IF','CL','CG','FF','FA','BR','CP', 
           'UP','CM','MI','1%','BO','GA','%P']] = player_df[['KI','MK','HB','DI','GL','BH',
                                                             'HO','TK','RB','IF','CL','CG',
                                                             'FF','FA','BR','CP','UP','CM',
                                                             'MI','1%','BO','GA','%P']].fillna(0)

### Separating off aggregate data from each season into different dataset

In [10]:
player_agg_df = player_df[player_df['opponent'] == 'Totals']

In [11]:
player_df = player_df.drop(player_agg_df.index)

In [12]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143904 entries, 0 to 154524
Data columns (total 32 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            143904 non-null  object 
 1   afl_tables_url  143904 non-null  object 
 2   team            143904 non-null  object 
 3   year            143904 non-null  object 
 4   game_num        143904 non-null  object 
 5   opponent        143904 non-null  object 
 6   Rd              143904 non-null  object 
 7   R               143904 non-null  object 
 8   #               143904 non-null  object 
 9   KI              143904 non-null  float64
 10  MK              143904 non-null  float64
 11  HB              143904 non-null  float64
 12  DI              143904 non-null  float64
 13  GL              143904 non-null  float64
 14  BH              143904 non-null  float64
 15  HO              143904 non-null  float64
 16  TK              143904 non-null  float64
 17  RB        

## Creating a player bio table

In [13]:
players_bio_list = []
for i, player_url in enumerate(player_df['afl_tables_url'].unique()):
#     print(i, player_url)
    page = requests.get('https://afltables.com/afl/stats' + player_url[2:])
    soup = BeautifulSoup(page.text)
    name = soup.find("h1").text
    height_and_weight = soup.text.split('\n')[8]
    birthday_debut = soup.text.split('\n')[7]
#     print('name', name, 'h and w', height_and_weight)
    players_bio_list.append((player_url, name, height_and_weight,birthday_debut))

In [14]:
player_table_data = []
for player in players_bio_list:
    try:
        url = player[0]
        name = player[1]
        height = re.split(' |:',player[2])[1]
        weight = re.split(' |:',player[2])[4]
        dob = re.split(':| |\(',player[3])[9]
        debut_list = [re.split(':| |\(',player[3])[12]]
        if re.split(':| |\(',player[3])[13] != 'Last':
            debut_list.append(re.split(':| |\(',player[3])[13])
            last_list = [re.split(':| |\(',player[3])[15]]
            try:
                last_list.append(re.split(':| |\(',player[3])[16])
            except:
                pass
        else:
            last_list = [re.split(':| |\(',player[3])[14]]
            try:
                last_list.append(re.split(':| |\(',player[3])[15])
            except:
                pass
        last = ' '.join(last_list)
        debut = ' '.join(debut_list)
        player_table_data.append((url, name, height, weight, dob, debut, last))
    except:
        print(player)
        break

In [15]:
player_data_df = pd.DataFrame(player_table_data, columns = ['afl_tables_url', 'name', 'height', 
                                                            'weight', 'dob', 'debut', 'last'])

In [16]:
player_data_df['height'] = player_data_df['height'].astype(int)
player_data_df['weight'] = player_data_df['weight'].astype(int)
player_data_df['dob'] = pd.to_datetime(player_data_df['dob'], format="%d-%b-%Y")

In [17]:
player_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1603 entries, 0 to 1602
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   afl_tables_url  1603 non-null   object        
 1   name            1603 non-null   object        
 2   height          1603 non-null   int32         
 3   weight          1603 non-null   int32         
 4   dob             1603 non-null   datetime64[ns]
 5   debut           1603 non-null   object        
 6   last            1603 non-null   object        
dtypes: datetime64[ns](1), int32(2), object(4)
memory usage: 75.3+ KB


In [18]:
player_data_df['debut_year'] = player_data_df['debut'].apply(lambda x: 0 \
        if len(re.findall(r'\d+(?=y)', x)) == 0 else re.findall(r'\d+(?=y)', x)[0]).astype(int)
player_data_df['debut_day'] = player_data_df['debut'].apply(lambda x: 0 \
        if len(re.findall(r'y (\d+)d', x)) == 0 else re.findall(r'y (\d+)d', x)[0].strip()).astype(int)

player_data_df['last_year'] = player_data_df['last'].apply(lambda x: 0 \
        if len(re.findall(r'\d+(?=y)', x)) == 0 else re.findall(r'\d+(?=y)', x)[0]).astype(int)
player_data_df['last_day'] = player_data_df['last'].apply(lambda x: 0 \
        if len(re.findall(r'y (\d+)d', x)) == 0 else re.findall(r'y (\d+)d', x)[0].strip()).astype(int)

In [19]:
today = datetime.today()
def get_date(row, column):
    return (today - relativedelta(years=row[f'{column}_year'], days=row[f'{column}_day'])).date()

In [20]:
player_data_df['debut_date'] = player_data_df.apply(lambda x: get_date(x, 'debut'), axis=1)
player_data_df['last_date'] = player_data_df.apply(lambda x: get_date(x, 'last'), axis=1)

In [21]:
player_data_df

,afl_tables_url,name,height,weight,dob,debut,last,debut_year,debut_day,last_year,last_day,debut_date,last_date
0,../players/J/Jonathan_Freeman.html,Jonathan Freeman,198,96,1994-04-27,20y 105d,22y 38d),20,105,22,38,2003-01-01,2001-03-09
1,../players/T/Travis_Colyer.html,Travis Colyer,175,77,1991-08-24,18y 229d,30y 339d),18,229,30,339,2004-08-30,1992-05-12
2,../players/T/Tom_Boyd.html,Tom Boyd,200,103,1995-08-22,18y 241d,22y 334d),18,241,22,334,2004-08-18,2000-05-17
3,../players/O/Oliver_Dempsey.html,Oliver Dempsey,185,70,2003-01-07,19y 107d,19y 113d),19,107,19,113,2003-12-31,2003-12-25
4,../players/B/Brayden_Ainsworth.html,Brayden Ainsworth,184,80,1998-11-27,19y 166d,23y 120d),19,166,23,120,2003-11-02,1999-12-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,../players/D/Dan_Robinson.html,Dan Robinson,184,85,1994-07-03,20y 330d,24y 67d),20,330,24,67,2002-05-21,1999-02-08
1599,../players/R/Ryan_Hargrave.html,Ryan Hargrave,190,87,1981-07-26,20y 248d,31y 17d),20,248,31,17,2002-08-11,1992-03-30
1600,../players/R/Richard_Tambling.html,Richard Tambling,180,80,1986-09-12,18y 231d,26y 353d),18,231,26,353,2004-08-28,1996-04-28
1601,../players/J/Jarrod_Pickett.html,Jarrod Pickett,178,77,1996-08-18,20y 217d,21y 359d),20,217,21,359,2002-09-11,2001-04-22


In [22]:
player_data_df = player_data_df.drop(['debut', 'last', 'debut_year', 'debut_day',
                                      'last_year', 'last_day'], axis=1)

## Creating a games stats table

In [23]:
game_df = pd.DataFrame(columns=['year','round','date_time', 'venue', 'home_team', 'away_team', 
                                'attendance', 'home_team_quarter_score', 'away_team_quarter_score', 
                                'home_team_final_score', 'away_team_final_score'])
game_df

,year,round,date_time,venue,home_team,away_team,attendance,home_team_quarter_score,away_team_quarter_score,home_team_final_score,away_team_final_score


In [24]:
round_list = ['Round 1','Round 2','Round 3','Round 4','Round 5','Round 6','Round 7','Round 8',
              'Round 9','Round 10','Round 11','Round 12','Round 13','Round 14','Round 15','Round 16',
              'Round 17','Round 18','Round 19','Round 20','Round 21','Round 22','Round 23',
              'Qualifying Final','Elimination Final','Semi Final','Preliminary Final','Grand Final']

year_list = sorted(list(player_df['year'].unique()))

In [25]:
for year in year_list:
    game_tables = pd.read_html(f'https://afltables.com/afl/seas/{year}.html')
    for game_table in game_tables:
        try:
            if len(game_table) == 1:
                if game_table[0][0] in round_list:
                    rnd_num = game_table[0][0]
                else:
                    continue
            elif len(game_table) == 2:
                home_team = game_table[0][0]
                away_team = game_table[0][1]
                home_team_quarter_scores = game_table[1][0]
                away_team_quarter_scores = game_table[1][1]
                home_team_final_score = game_table[2][0]
                away_team_final_score = game_table[2][1]
                d = re.split(' Att: | Venue: ',game_table[3][0])
                if len(d) == 3:
                    date_time = d[0]
                    attendance = d[1]
                    venue = d[2]
                elif len(d) == 2:
                    date_time = d[0]
                    attendance = 0
                    venue = d[1]
                else:
                    raise Exception(f'Unexpected data {d}')
                created_row = (year, rnd_num, date_time, venue, home_team, away_team, attendance,
                               home_team_quarter_scores, away_team_quarter_scores, 
                               home_team_final_score, away_team_final_score)
                game_df.loc[len(game_df)] = created_row
            else:
                continue
        except Exception as e:
            print(f'{year}, {game_table}')
            raise Exception(e)

In [26]:
game_df = game_df[~game_df['home_team_final_score'].isna()]

In [27]:
game_df[['day_of_week','date','AEST_time',
         'AEST_am_pm','local_time','local_am_pm']] = game_df['date_time'].str.split(' ',expand=True)
game_df = game_df.drop('date_time', axis=1)

C:\Users\aaron\AppData\Local\Temp\ipykernel_9768\1222839064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df[['day_of_week','date','AEST_time',
C:\Users\aaron\AppData\Local\Temp\ipykernel_9768\1222839064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df[['day_of_week','date','AEST_time',
C:\Users\aaron\AppData\Local\Temp\ipykernel_9768\1222839064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [28]:
game_df['date'] = pd.to_datetime(game_df['date'], format="%d-%b-%Y")
game_df['local_time'] = game_df['local_time'].apply(lambda x: x[1:] if x else x)
game_df['local_am_pm'] = game_df['local_am_pm'].apply(lambda x: x[:-1] if x else x)

In [29]:
game_df['AEST'] = pd.to_datetime(game_df['AEST_time']+' '+game_df['AEST_am_pm'], format="%I:%M %p").dt.time
game_df['local'] = pd.to_datetime(game_df['local_time']+' '+game_df['local_am_pm'], format="%I:%M %p").dt.time

In [30]:
game_df = game_df.drop(['AEST_time', 'AEST_am_pm', 'local_time', 'local_am_pm'], axis=1)

In [31]:
game_df['local'] = game_df['local'].fillna(game_df['AEST'])

In [32]:
game_df['home_team_quarter_score'].str.split(' ')

0        [5.4, 7.9, 12.13, 13.17]
1       [5.5, 13.8, 18.15, 24.22]
2         [4.2, 9.6, 15.8, 17.10]
3         [2.1, 3.3, 10.7, 13.15]
4        [6.2, 10.5, 15.9, 20.16]
                  ...            
5750       [3.2, 4.6, 7.10, 9.10]
5751     [3.2, 8.6, 13.10, 17.11]
5752     [2.8, 3.13, 8.19, 11.22]
5753         [2.2, 4.4, 6.8, 9.9]
5754      [3.2, 4.3, 14.8, 19.13]
Name: home_team_quarter_score, Length: 5755, dtype: object

In [33]:
game_df[['home_Q1', 'home_Q2', 'home_Q3', 'home_Q4', 'home_extra_time']] = game_df['home_team_quarter_score'].str.split(re.compile(r'[ |\xa0]+'), regex=True, expand=True)
game_df[['away_Q1', 'away_Q2', 'away_Q3', 'away_Q4', 'away_extra_time']] = game_df['away_team_quarter_score'].str.split(re.compile(r'[ |\xa0]+'), regex=True, expand=True)

game_df = game_df.drop(['home_team_quarter_score','away_team_quarter_score'], axis=1)

In [34]:
game_df['round'] = game_df['round'].replace({'Qualifying Final':'QF','Elimination Final':'EF',
                                             'Semi Final':'SF','Preliminary Final':'PF','Grand Final':'GF'})
game_df['round'] = game_df['round'].str.replace('Round ', '')

In [35]:
player_data_df

,afl_tables_url,name,height,weight,dob,debut_date,last_date
0,../players/J/Jonathan_Freeman.html,Jonathan Freeman,198,96,1994-04-27,2003-01-01,2001-03-09
1,../players/T/Travis_Colyer.html,Travis Colyer,175,77,1991-08-24,2004-08-30,1992-05-12
2,../players/T/Tom_Boyd.html,Tom Boyd,200,103,1995-08-22,2004-08-18,2000-05-17
3,../players/O/Oliver_Dempsey.html,Oliver Dempsey,185,70,2003-01-07,2003-12-31,2003-12-25
4,../players/B/Brayden_Ainsworth.html,Brayden Ainsworth,184,80,1998-11-27,2003-11-02,1999-12-18
...,...,...,...,...,...,...,...
1598,../players/D/Dan_Robinson.html,Dan Robinson,184,85,1994-07-03,2002-05-21,1999-02-08
1599,../players/R/Ryan_Hargrave.html,Ryan Hargrave,190,87,1981-07-26,2002-08-11,1992-03-30
1600,../players/R/Richard_Tambling.html,Richard Tambling,180,80,1986-09-12,2004-08-28,1996-04-28
1601,../players/J/Jarrod_Pickett.html,Jarrod Pickett,178,77,1996-08-18,2002-09-11,2001-04-22


In [36]:
player_df

,name,afl_tables_url,team,year,game_num,opponent,Rd,R,#,KI,...,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P
0,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,1.0,Adelaide,20,L,34,2.0,...,0.0,0.0,5.0,2.0,1.0,1.0,2.0,0.0,0.0,94.0
1,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,2.0,Collingwood,21,W,34,5.0,...,1.0,0.0,3.0,5.0,2.0,2.0,0.0,0.0,0.0,95.0
2,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,3.0,Fremantle,22,L,34,2.0,...,1.0,0.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,93.0
3,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,4.0,Geelong,23,L,34,2.0,...,0.0,0.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,95.0
5,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2015,5.0,Geelong,18,L,34,4.0,...,0.0,0.0,2.0,4.0,2.0,4.0,1.0,0.0,1.0,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154519,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2019,148.0,Western Bulldogs,14,W,20,6.0,...,0.0,0.0,7.0,7.0,2.0,3.0,0.0,0.0,1.0,77.0
154520,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2019,149.0,North Melbourne,15,L,20,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
154521,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2019,150.0,Greater Western Sydney,PF,L,20,3.0,...,2.0,0.0,6.0,0.0,2.0,2.0,3.0,0.0,0.0,86.0
154523,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2020,151.0,Sydney,10,W,20,5.0,...,0.0,0.0,5.0,4.0,0.0,0.0,1.0,0.0,0.0,89.0


In [37]:
game_df

,year,round,venue,home_team,away_team,attendance,home_team_final_score,away_team_final_score,day_of_week,date,...,home_Q1,home_Q2,home_Q3,home_Q4,home_extra_time,away_Q1,away_Q2,away_Q3,away_Q4,away_extra_time
0,1993,1,M.C.G.,Footscray,Collingwood,"58,997",95.0,115.0,Fri,1993-03-26,...,5.4,7.9,12.13,13.17,None,3.4,9.7,15.8,17.13,None
1,1993,1,M.C.G.,North Melbourne,Brisbane Bears,"8,653",166.0,143.0,Sat,1993-03-27,...,5.5,13.8,18.15,24.22,None,5.1,9.5,15.8,22.11,None
2,1993,1,Princes Park,Carlton,Fitzroy,"23,433",112.0,118.0,Sat,1993-03-27,...,4.2,9.6,15.8,17.10,None,5.4,6.10,10.14,17.16,None
3,1993,1,Waverley Park,Hawthorn,Melbourne,"25,098",93.0,70.0,Sat,1993-03-27,...,2.1,3.3,10.7,13.15,None,1.0,4.3,7.4,11.4,None
4,1993,1,Kardinia Park,Geelong,St Kilda,"24,337",136.0,112.0,Sat,1993-03-27,...,6.2,10.5,15.9,20.16,None,5.4,7.9,12.13,16.16,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5750,2023,4,S.C.G.,Sydney,Port Adelaide,"31,686",64.0,66.0,Sat,2023-04-08,...,3.2,4.6,7.10,9.10,None,0.3,4.6,5.8,9.12,None
5751,2023,4,Docklands,St Kilda,Gold Coast,"21,049",113.0,60.0,Sat,2023-04-08,...,3.2,8.6,13.10,17.11,None,3.4,4.8,5.11,8.12,None
5752,2023,4,Docklands,Essendon,Greater Western Sydney,"28,815",88.0,75.0,Sun,2023-04-09,...,2.8,3.13,8.19,11.22,None,4.0,6.5,9.7,11.9,None
5753,2023,4,Perth Stadium,West Coast,Melbourne,"42,458",63.0,126.0,Sun,2023-04-09,...,2.2,4.4,6.8,9.9,None,5.1,8.4,14.9,19.12,None


## Creating player position and draft data

In [38]:
combined_df=player_df.drop('name',axis=1).merge(player_data_df, on='afl_tables_url')

In [39]:
combined_df['team'].unique()


array(['Brisbane Lions', 'Essendon', 'Fremantle',
       'Greater Western Sydney', 'Western Bulldogs', 'Geelong',
       'West Coast', 'Gold Coast', 'Melbourne', 'Port Adelaide',
       'Richmond', 'Adelaide', 'Collingwood', 'Sydney', 'Kangaroos',
       'North Melbourne', 'St Kilda', 'Carlton', 'Hawthorn'], dtype=object)

In [45]:
footywire_map = {'Brisbane Lions':'brisbane-lions', 'Port Adelaide':'port-adelaide-power', 
                 'St Kilda':'st-kilda-saints', 'Geelong':'geelong-cats',
                 'Fremantle':'fremantle-dockers', 'Hawthorn':'hawthorn-hawks', 
                 'Gold Coast':'gold-coast-suns', 'Essendon':'essendon-bombers', 
                 'Carlton':'carlton-blues', 'Adelaide':'adelaide-crows', 
                 'North Melbourne':'kangaroos', 'West Coast':'west-coast-eagles', 
                 'Collingwood':'collingwood-magpies', 'Greater Western Sydney':'greater-western-sydney-giants', 
                 'Sydney':'sydney-swans', 'Melbourne':'melbourne-demons', 'Kangaroos':'kangaroos',
                 'Western Bulldogs':'western-bulldogs', 'Richmond':'richmond-tigers'}

footywire_name_map = {'jack':'jackson', 'ollie':'oliver', 'zach':'zachary', 'josh':'joshua', 'lachie':'lachlan', 
                      'dom':'dominic', 'tom':'thomas', 'cam':'cameron', 'nick':'nicholas', 'tim':'timothy',
                      'will':'william', 'mitch':'mitchell', 'matt':'matthew', 'brad':'bradley', 'alex':'alexis',
                      'sam':'samuel', 'timmy':'tim', 'harry':'harrison', 'matthew':'matt', 'brett':'bret', 
                      'zac':'zachary', 'dan':'daniel', 'ed':'edward', 'steve':'stephen', 'bobby':'robert', 
                      'robbie':'robert', 'ned':'edward', 'pat':'patrick', 'nic':'nicholas', 'junior':'willie',
                      'nik':'nicholas', 'eddie':'edward', 'jeff':'jeffrey', 'paddy':'patrick', 'nat':'nathan',
                      'ben':'benjamin', 'chris':'christopher', 'ricky':'rick', 'ty':'tyrone', 'seb':'sebastian',
                      'jimmy':'james', 'jayden':'jaydyn', 'niall':'nial'}

url_map = {'pp-fremantle-dockers--michael-frederick':'pp-fremantle-dockers--minairo-frederick',
            'pp-greater-western-sydney-giants--robert-hill':'pp-greater-western-sydney-giants--ian-hill',
            'pp-adelaide-crows--elliott-himmelberg':'pp-adelaide-crows--elliot-himmelberg',
            'pp-greater-western-sydney-giants--zachary-langdon':'pp-west-coast-eagles--zac-giles-langdon',
            'pp-essendon-bombers--nicholas-cox':'pp-essendon-bombers--nikolas-cox',
            'pp-hawthorn-hawks--angus-dewar':'pp-west-coast-eagles--angus-litherland',
            'pp-west-coast-eagles--zachary-langdon':'pp-west-coast-eagles--zac-giles-langdon',
            'pp-essendon-bombers--cory-dellolio':'pp-essendon-bombers--corey-dell-olio',
            'pp-st-kilda-saints--mitchell-o-wens':'pp-st-kilda-saints--mitchito-owens',
            'pp-essendon-bombers--massimo-dambrosio':'pp-essendon-bombers--massimo-d-ambrosio',
            'pp-carlton-blues--joshua-deluca':'pp-carlton-blues--josh-deluca-cardillo',
            'pp-kangaroos--lachlan-hosie':'pp-kangaroos--lachie-hosie',
            'pp-fremantle-dockers--joshua-deluca':'pp-carlton-blues--josh-deluca-cardillo',
            'pp-collingwood-magpies--robert-hill':'pp-collingwood-magpies--ian-hill',
            'pp-fremantle-dockers--jason-carter':'pp-fremantle-dockers--jason-carter-1',
            'pp-geelong-cats--oliver-dempsey':'pp-geelong-cats--ollie-dempsey',
            'pp-west-coast-eagles--angus-dewar':'pp-west-coast-eagles--angus-litherland',
            'pp-st-kilda-saints--beau-maister':'pp-st-kilda-saints--beau-wilkes',
            'pp-collingwood-magpies--heritier-lumumba':'pp-melbourne-demons--heritier-o-brien',
            'pp-western-bulldogs--brian-lake':'pp-hawthorn-hawks--brian-harris',
            'pp-fremantle-dockers--matthew-de-boer':'pp-greater-western-sydney-giants--matthew-deboer',
            'pp-port-adelaide-power--jasper-pittard':'pp-kangaroos--jasper-mcmillan-pittard',
            'pp-carlton-blues--jeffrey-garlett':'pp-melbourne-demons--jeff-gartlett',
            'pp-west-coast-eagles--beau-maister':'pp-st-kilda-saints--beau-wilkes',
            'pp-greater-western-sydney-giants--matthew-de-boer':'pp-greater-western-sydney-giants--matthew-deboer',
            'pp-melbourne-demons--jeffrey-garlett':'pp-melbourne-demons--jeff-gartlett',           
            'pp-hawthorn-hawks--brian-lake':'pp-hawthorn-hawks--brian-harris',
            'pp-kangaroos--jasper-pittard':'pp-kangaroos--jasper-mcmillan-pittard',
            'pp-melbourne-demons--heritier-lumumba':'pp-melbourne-demons--heritier-o-brien',
            'pp-port-adelaide-power--simon-phillips':'pp-port-adelaide-power--simon-phillips-1',
            'pp-sydney-swans--simon-phillips':'pp-port-adelaide-power--simon-phillips-1',
            'pp-sydney-swans--kieren-jackson':'pp-sydney-swans--kieran-jack'}

def get_footywire_url(team, player_name):
    return f'https://www.footywire.com/afl/footy/pp-{footywire_map[team]}--{player_name}'

In [46]:
extra_player_data_list = []
count = 0
for afl_tables_url, name, team in list(combined_df[['afl_tables_url', 'name', 'team']].value_counts().index):
    count += 1
    if count % 100 == 0:
        print(count)
    flag = None
    player_name = name.lower().replace(' ', '-')
    while True:
        url_string = get_footywire_url(team, player_name)
        try:
            page = requests.get(url_string)
        except:
            print(afl_tables_url, name, team, url_string)
        soup = BeautifulSoup(page.text)
        if not soup.find(id="playerProfileData"):
            if not flag:
                player_name = '-'.join([footywire_name_map.get(n,n) for n in name.lower().split(' ')])
                flag = 1
            elif flag == 1:
                flag = 2
                if name.lower().split(' ')[-1][0] != 'o':
                    flag = 4
                    break
                name = ' '.join(name.lower().split(' ')[:-1]) + ' ' + name.lower().split(' ')[-1][0] \
                            + ' ' + name.lower().split(' ')[-1][1:]
                player_name = '-'.join([n for n in name.lower().split(' ')])
            elif flag == 2:
                player_name = '-'.join([footywire_name_map.get(n,n) for n in name.lower().split(' ')])
                flag=3
            else:
                flag = 4
                break
        else:
            break
    if flag == 4:
        player_name = '-'.join([footywire_name_map.get(n,n) for n in name.lower().split(' ')])
        url_string = get_footywire_url(team, player_name)
        url_string = '/'.join(url_string.split('/')[:-1])+'/'+url_map.get(url_string.split('/')[-1], url_string.split('/')[-1])
        try:
            page = requests.get(url_string)
        except:
            print(afl_tables_url, name, team, url_string)
        soup = BeautifulSoup(page.text)
        if not soup.find(id="playerProfileData"):
            print(afl_tables_url, name, team, url_string)
            continue
    try:
        if soup.find(id="playerProfileTeamDiv"):
            team_data = soup.find(id="playerProfileTeamDiv").find('b').text.replace('\xa0', '').replace('\n', '')
        else:
            team_data = None
        if soup.find(id="playerProfileData1"):
            data_1 = soup.find(id="playerProfileData1").text.replace('\xa0', '').replace('\n', ' ')
        else:
            data_1 = None
        if soup.find(id="playerProfileData2"):    
            data_2 = soup.find(id="playerProfileData2").text.replace('\xa0', '').replace('\n', ' ')
        else:
            data_2 = None
        if soup.find(id="playerProfileDraftInfo"):
            draft_data = soup.find(id="playerProfileDraftInfo").text.replace('\xa0', '').replace('\n', ' ')
        else:
            draft_data = None
    except AttributeError:
        print(afl_tables_url, name, team, url_string)
        continue
    extra_player_data_list.append((afl_tables_url, team, team_data, data_1, data_2, draft_data))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [47]:
extra_df = pd.DataFrame(extra_player_data_list, columns=['afl_tables_url', 'team', 'team_data', 
                                                         'data_1', 'data_2', 'draft_data'])

In [48]:
extra_df[['misc', 'origin']] = extra_df['data_1'].str.split('Origin: ', expand=True)
extra_df[['misc2', 'position']] = extra_df['data_2'].str.split('Position: ', expand=True)
extra_df['draft_round'] = extra_df['draft_data'].str.extract(r'Round\s(\d+)\s*,', expand=False).astype(float)
extra_df['draft_pick_number'] = extra_df['draft_data'].str.extract(r'#(\d+)* ', expand=False).astype(float)
extra_df['draft_year'] = extra_df['draft_data'].str.extract(r'Last Drafted:.*(\d{4})', expand=False)
extra_df['extra_draft_data'] = extra_df['draft_data'].str.extract(r'Last Drafted:.*\d{4}.*?(.*)', expand=True)
extra_df[['draft_type', 'draft_team']] = extra_df['extra_draft_data'].str.split(' by ', expand=True)

extra_df = extra_df.drop(['team_data', 'data_1', 'data_2', 'draft_data', 'misc', 
                          'misc2', 'extra_draft_data'], axis=1)
extra_df

,afl_tables_url,team,origin,position,draft_round,draft_pick_number,draft_year,draft_type,draft_team
0,../players/D/Dustin_Fletcher.html,Essendon,Western Jets,Defender,NaN,3.0,1992,Pre-Draft Selections Draft,Essendon Bombers
1,../players/D/David_Mundy.html,Fremantle,Murray Bushrangers,Midfield,2.0,19.0,2003,National Draft,Fremantle Dockers
2,../players/A/Adam_Goodes.html,Sydney,North Ballarat Rebels,Forward,3.0,43.0,1997,National Draft,Sydney Swans
3,../players/S/Scott_Pendlebury.html,Collingwood,Gippsland Power,Midfield,1.0,5.0,2005,National Draft,Collingwood Magpies
4,../players/J/Joel_Selwood.html,Geelong,Bendigo,Midfield,1.0,7.0,2006,National Draft,Geelong Cats
...,...,...,...,...,...,...,...,...,...
2214,../players/R/Ryan_Angwin.html,Greater Western Sydney,Gippsland Power,"Midfield, Forward ...",1.0,18.0,2020,National Draft,GWS Giants
2215,../players/R/Ryan_Neates.html,West Coast,Claremont,Midfield,1.0,5.0,2009,Pre-Season Draft,West Coast Eagles
2216,../players/C/Craig_Moller.html,Fremantle,Carlton,Ruck,5.0,46.0,2012,Rookie Draft,Fremantle Dockers
2217,../players/R/Rhys_Palmer.html,Carlton,East Fremantle,Forward,1.0,7.0,2007,National Draft,Fremantle Dockers


In [49]:
temp_df = pd.DataFrame(list(extra_df[['afl_tables_url', 'origin', 'draft_round', 'draft_pick_number',
                                      'draft_year', 'draft_type', 'draft_team']].value_counts().index), 
                      columns=['afl_tables_url', 'origin', 'draft_round', 'draft_pick_number',
                                      'draft_year', 'draft_type', 'draft_team'])

In [50]:
temp_df2 = pd.DataFrame(list(extra_df[['afl_tables_url', 'team', 'position']].value_counts().index), 
                        columns=['afl_tables_url', 'team', 'position'])

In [51]:
player_df_final = player_df.merge(temp_df2, how='left', on=['afl_tables_url','team'])

In [52]:
player_data_df_final = player_data_df.merge(temp_df, how='left', on='afl_tables_url')

## Exporting

In [53]:
player_df.to_csv('afltables_stats')

In [54]:
player_agg_df.to_csv('afltables_agg_stats')

In [55]:
player_data_df.to_csv('afltables_player')

In [56]:
game_df.to_csv('afltables_game')

In [57]:
extra_df.to_csv('extra_bio_info')

In [58]:
player_data_df_final.to_csv('afltables_player_final')

In [59]:
player_df_final.to_csv('afltables_stats_final')

## Final dfs

In [60]:
game_df

,year,round,venue,home_team,away_team,attendance,home_team_final_score,away_team_final_score,day_of_week,date,...,home_Q1,home_Q2,home_Q3,home_Q4,home_extra_time,away_Q1,away_Q2,away_Q3,away_Q4,away_extra_time
0,1993,1,M.C.G.,Footscray,Collingwood,"58,997",95.0,115.0,Fri,1993-03-26,...,5.4,7.9,12.13,13.17,None,3.4,9.7,15.8,17.13,None
1,1993,1,M.C.G.,North Melbourne,Brisbane Bears,"8,653",166.0,143.0,Sat,1993-03-27,...,5.5,13.8,18.15,24.22,None,5.1,9.5,15.8,22.11,None
2,1993,1,Princes Park,Carlton,Fitzroy,"23,433",112.0,118.0,Sat,1993-03-27,...,4.2,9.6,15.8,17.10,None,5.4,6.10,10.14,17.16,None
3,1993,1,Waverley Park,Hawthorn,Melbourne,"25,098",93.0,70.0,Sat,1993-03-27,...,2.1,3.3,10.7,13.15,None,1.0,4.3,7.4,11.4,None
4,1993,1,Kardinia Park,Geelong,St Kilda,"24,337",136.0,112.0,Sat,1993-03-27,...,6.2,10.5,15.9,20.16,None,5.4,7.9,12.13,16.16,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5750,2023,4,S.C.G.,Sydney,Port Adelaide,"31,686",64.0,66.0,Sat,2023-04-08,...,3.2,4.6,7.10,9.10,None,0.3,4.6,5.8,9.12,None
5751,2023,4,Docklands,St Kilda,Gold Coast,"21,049",113.0,60.0,Sat,2023-04-08,...,3.2,8.6,13.10,17.11,None,3.4,4.8,5.11,8.12,None
5752,2023,4,Docklands,Essendon,Greater Western Sydney,"28,815",88.0,75.0,Sun,2023-04-09,...,2.8,3.13,8.19,11.22,None,4.0,6.5,9.7,11.9,None
5753,2023,4,Perth Stadium,West Coast,Melbourne,"42,458",63.0,126.0,Sun,2023-04-09,...,2.2,4.4,6.8,9.9,None,5.1,8.4,14.9,19.12,None


In [61]:
player_data_df_final

,afl_tables_url,name,height,weight,dob,debut_date,last_date,origin,draft_round,draft_pick_number,draft_year,draft_type,draft_team
0,../players/J/Jonathan_Freeman.html,Jonathan Freeman,198,96,1994-04-27,2003-01-01,2001-03-09,Aspley,4.0,62.0,2013,National Draft,Brisbane Lions
1,../players/T/Travis_Colyer.html,Travis Colyer,175,77,1991-08-24,2004-08-30,1992-05-12,Claremont,2.0,26.0,2009,National Draft,Essendon Bombers
2,../players/T/Tom_Boyd.html,Tom Boyd,200,103,1995-08-22,2004-08-18,2000-05-17,Eastern Ranges,1.0,1.0,2013,National Draft,GWS Giants
3,../players/O/Oliver_Dempsey.html,Oliver Dempsey,185,70,2003-01-07,2003-12-31,2003-12-25,Carey Grammar,1.0,15.0,2021,Rookie Draft,Geelong Cats
4,../players/B/Brayden_Ainsworth.html,Brayden Ainsworth,184,80,1998-11-27,2003-11-02,1999-12-18,Subiaco,2.0,32.0,2017,National Draft,West Coast Eagles
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,../players/D/Dan_Robinson.html,Dan Robinson,184,85,1994-07-03,2002-05-21,1999-02-08,Geelong,6.0,50.0,2012,Rookie Draft,Sydney Swans
1599,../players/R/Ryan_Hargrave.html,Ryan Hargrave,190,87,1981-07-26,2002-08-11,1992-03-30,Perth,5.0,66.0,1999,National Draft,Western Bulldogs
1600,../players/R/Richard_Tambling.html,Richard Tambling,180,80,1986-09-12,2004-08-28,1996-04-28,Southern Districts,1.0,4.0,2004,National Draft,Richmond Tigers
1601,../players/J/Jarrod_Pickett.html,Jarrod Pickett,178,77,1996-08-18,2002-09-11,2001-04-22,South Fremantle,1.0,4.0,2014,National Draft,GWS Giants


In [62]:
player_df_final

,name,afl_tables_url,team,year,game_num,opponent,Rd,R,#,KI,...,BR,CP,UP,CM,MI,1%,BO,GA,%P,position
0,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,1.0,Adelaide,20,L,34,2.0,...,0.0,5.0,2.0,1.0,1.0,2.0,0.0,0.0,94.0,Forward
1,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,2.0,Collingwood,21,W,34,5.0,...,0.0,3.0,5.0,2.0,2.0,0.0,0.0,0.0,95.0,Forward
2,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,3.0,Fremantle,22,L,34,2.0,...,0.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,93.0,Forward
3,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,4.0,Geelong,23,L,34,2.0,...,0.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,95.0,Forward
4,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2015,5.0,Geelong,18,L,34,4.0,...,0.0,2.0,4.0,2.0,4.0,1.0,0.0,1.0,84.0,Forward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143899,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2019,148.0,Western Bulldogs,14,W,20,6.0,...,0.0,7.0,7.0,2.0,3.0,0.0,0.0,1.0,77.0,Forward
143900,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2019,149.0,North Melbourne,15,L,20,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,Forward
143901,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2019,150.0,Greater Western Sydney,PF,L,20,3.0,...,0.0,6.0,0.0,2.0,2.0,3.0,0.0,0.0,86.0,Forward
143902,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2020,151.0,Sydney,10,W,20,5.0,...,0.0,5.0,4.0,0.0,0.0,1.0,0.0,0.0,89.0,Forward


In [63]:
player_data_df

,afl_tables_url,name,height,weight,dob,debut_date,last_date
0,../players/J/Jonathan_Freeman.html,Jonathan Freeman,198,96,1994-04-27,2003-01-01,2001-03-09
1,../players/T/Travis_Colyer.html,Travis Colyer,175,77,1991-08-24,2004-08-30,1992-05-12
2,../players/T/Tom_Boyd.html,Tom Boyd,200,103,1995-08-22,2004-08-18,2000-05-17
3,../players/O/Oliver_Dempsey.html,Oliver Dempsey,185,70,2003-01-07,2003-12-31,2003-12-25
4,../players/B/Brayden_Ainsworth.html,Brayden Ainsworth,184,80,1998-11-27,2003-11-02,1999-12-18
...,...,...,...,...,...,...,...
1598,../players/D/Dan_Robinson.html,Dan Robinson,184,85,1994-07-03,2002-05-21,1999-02-08
1599,../players/R/Ryan_Hargrave.html,Ryan Hargrave,190,87,1981-07-26,2002-08-11,1992-03-30
1600,../players/R/Richard_Tambling.html,Richard Tambling,180,80,1986-09-12,2004-08-28,1996-04-28
1601,../players/J/Jarrod_Pickett.html,Jarrod Pickett,178,77,1996-08-18,2002-09-11,2001-04-22


In [64]:
player_df

,name,afl_tables_url,team,year,game_num,opponent,Rd,R,#,KI,...,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P
0,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,1.0,Adelaide,20,L,34,2.0,...,0.0,0.0,5.0,2.0,1.0,1.0,2.0,0.0,0.0,94.0
1,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,2.0,Collingwood,21,W,34,5.0,...,1.0,0.0,3.0,5.0,2.0,2.0,0.0,0.0,0.0,95.0
2,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,3.0,Fremantle,22,L,34,2.0,...,1.0,0.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,93.0
3,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2014,4.0,Geelong,23,L,34,2.0,...,0.0,0.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,95.0
5,Jonathan Freeman,../players/J/Jonathan_Freeman.html,Brisbane Lions,2015,5.0,Geelong,18,L,34,4.0,...,0.0,0.0,2.0,4.0,2.0,4.0,1.0,0.0,1.0,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154519,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2019,148.0,Western Bulldogs,14,W,20,6.0,...,0.0,0.0,7.0,7.0,2.0,3.0,0.0,0.0,1.0,77.0
154520,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2019,149.0,North Melbourne,15,L,20,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
154521,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2019,150.0,Greater Western Sydney,PF,L,20,3.0,...,2.0,0.0,6.0,0.0,2.0,2.0,3.0,0.0,0.0,86.0
154523,Ben Reid,../players/B/Ben_Reid.html,Collingwood,2020,151.0,Sydney,10,W,20,5.0,...,0.0,0.0,5.0,4.0,0.0,0.0,1.0,0.0,0.0,89.0
